# 어휘 집합 구축하기 
HuggingFace 기반의 tokenizers 라이브러리를 활용해 BPE 기반의 토크나이저를 구축하는 코드입니다. 우선 어휘집합부터 구축해야 합니다. 

In [2]:
#패키지 설치 
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#말뭉치 받기 및 전처리 
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download = True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 145MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 76.8MB/s]


In [6]:
#NSMC에 포함된 영화 리뷰를 순수 텍스트 형태로 코랩 환경 로컬의 지정된 디렉터리에 저장
import os

def write_lines(path, lines):
  with open(path, 'w', encoding = 'utf-8')as f:
    for line in lines:
      f.write(f'{line}\n')
write_lines("/content/train.txt",nsmc.train.get_all_texts())
write_lines("/content/test.txt",nsmc.test.get_all_texts())


#GPT 토크나이저 구축 
GPT 계열 모델이 사용하는 토크나이저 = BPE   
유니코드 바이트 수준으로 어휘 집합 구축, 토큰화 진행 

In [7]:
#GPT 토크나이저 구축 
#디렉토리 만들기 
import os
os.makedirs("/content/drive/MyDrive/nlpbook/bbpe", exist_ok = True)

In [10]:
#바이트 수준 BPE 어휘 집합 구축 
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files = ["/content/train.txt","/content/test.txt"],
    vocab_size = 10000,  #어휘집합 크기 설정
    special_tokens = ["[PAD]"] #특수 토큰 추가 
)
bytebpe_tokenizer.save_model("/content/drive/MyDrive/nlpbook/bbpe")

['/content/drive/MyDrive/nlpbook/bbpe/vocab.json',
 '/content/drive/MyDrive/nlpbook/bbpe/merges.txt']

#BERT 토크나이저 
BERT는 wordpiece 토크나이저를 사용합니다.  
BDE와 달리 vocab 만 활용하기 때문에 vacab.txt만 생성됩니다.  

In [11]:
#BERT 토크나이저 구축 
import os
os.makedirs('/content/drive/MyDrive/nlpbook/wordpiece', exist_ok=True)

In [13]:
#BERT 모델이 사용하는 워드피스 어휘 집합 구축 
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase = False)
wordpiece_tokenizer.train(
    files = ["/content/train.txt", "/content/test.txt"],
    vocab_size = 10000,
)
wordpiece_tokenizer.save_model("/content/drive/MyDrive/nlpbook/wordpiece")

['/content/drive/MyDrive/nlpbook/wordpiece/vocab.txt']